In [1]:
import pickle

In [2]:
import os
from tqdm import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
dataset_name = 'wnrr'
entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
entity_strings_set = set(entity_strings)
len(entity_strings_set)

40943

In [5]:
data = {}
splits = ['train', 'valid', 'test']
dataset_name = 'wnrr'
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

100%|█████████████████████████████████████████████████████████| 6268/6268 [00:00<00:00, 656804.09it/s]


In [6]:
filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

In [26]:
# fname = 'scores.pickle'
# fname = 'scores/scores_full_codexm_small.pickle'
fname = 'scores/scores_wnrr_small_ptlm.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [27]:
scores_data['prediction_strings'][0]

['diuril, NN -- a diuretic drug (trade name Diuril) used in the treatment of edema and hypertension',
 'triavil, NN -- tranquilizer and antidepressant (trade name Triavil) sometimes used as an antiemetic for adults',
 'nadolol, NN -- a beta-adrenergic blocking agent (trade name Corgard) that is used to treat hypertension and angina',
 'oradexon, NN -- a corticosteroid drug (trade names Decadron or Dexamethasone Intensol or Dexone or Hexadrol or Oradexon) used to treat allergies or inflammation',
 'quinora, NN -- cardiac drug (trade names Quinidex and Quinora) used to treat certain heart arrhythmias',
 'phensuximide, NN -- anticonvulsant (trade name Milontin) used to treat petit mal',
 'timolol, NN -- a beta blocker (trade name Blocadren) administered after heart attacks',
 'tadalafil, NN -- virility drug (trade name Cialis) used to treat erectile dysfunction in men',
 'mevacor, NN -- an oral drug (trade name Mevacor) to reduce blood cholesterol levels',
 'ciprofloxacin, NN -- an oral a

In [28]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

6268it [00:00, 11964.75it/s]


In [29]:
max([len(predictions_scores_dicts[x]) for x in range(500)])

130

In [30]:
id = 0
print(scores_data['input_strings'][id], '|||', scores_data['target_strings'][id])
# print(scores_data['scores'][2])
predictions_scores_dicts[id]

predict tail: trade name, NN -- a name given to a product or service | member of domain usage ||| metharbital, NN -- anticonvulsant drug (trade name Gemonil) used in the treatment of epilepsy


defaultdict(list,
            {'nitrofurantoin, NN -- derivative of nitrofuran used as an antibacterial medicine (trade name Macrodantin) effective against a broad range of Gram-positive and Gram-negative bacteria': -5.1356378,
             'st. joseph, NN -- the acetylated derivative of salicylic acid': -4.249753,
             'polaroid, NN -- (trade mark) a plastic film that can polarize a beam of light': -5.194192,
             'tolbutamide, NN -- sulfonylurea': -5.237251,
             'pamelor, NN -- a tricyclic antidepressant drug (trade name Pamelor) used along with psychotherapy to treat dysthymic depression': -5.65048,
             'methyldopa, NN -- antihypertensive drug (trade name Aldomet) used in the treatment of high blood pressure': -5.041038,
             'zimmer frame, NN -- a light enclosing framework (trade name Zimmer) with rubber castors or wheels and handles': -4.926204,
             'terbinafine, NN -- an oral antifungal drug (trade name Lamisil) used to treat cas

In [31]:
len(predictions_scores_dicts)

6268

In [32]:
import numpy as np
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0
hits_at_all = 0
count = 0
for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
    if len(filtering_entities) == 1:
        count += 1
    if 'head' in inputs:
        head_num_filter += len(filtering_entities)
    else:
        tail_num_filter += len(filtering_entities)
    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
        hits_at_all += 1
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)
print(head_num_filter/len(predictions_filtered), tail_num_filter/len(predictions_filtered))
print(hits_at_all/len(predictions_filtered))
print(count)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 6268/6268 [00:00<00:00, 25098.50it/s]

12.832322910019144 3.163848117421825
0.566847479259732
2753


In [34]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
count2 = 0
total_count = 0
for i in range(len(predictions_filtered)):
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
#     if len(filtering_entities) == 1:
#         total_count += 1
#     else:
#         continue
    total_count += 1
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 3 and target not in preds:
        num_small_arrs += 1
    if target in preds:
        count2 += 1
        
if total_count ==0:
    total_count = len(predictions_filtered)
for k in k_list:
    hits_at_k = count[k]/total_count
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/total_count)
print(num_small_arrs/total_count, 'were <3 length preds array without answer')
print(total_count, len(predictions_filtered))

hits@1 0.5103701340140395
hits@3 0.5520102105934908
hits@10 0.5660497766432674
mrr 0.5317186401695957
0.1080089342693044 were <3 length preds array without answer
6268 6268


In [35]:
count2/len(predictions_filtered)

0.566847479259732

In [44]:
id = 4
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]

In [45]:
print(inputs, 'Target:', target)
preds[:10], target

predict tail: united kingdom of great britain and northern ireland, NN -- a monarchy in northwestern Europe occupying most of the British Isles | member of domain region Target: facer, NN -- (a dated Briticism) a serious difficulty with which one is suddenly faced


([('twaddle, NN -- pretentious or silly talk or writing', -7.224674224853516),
  ('oxbridge, NN -- general term for an ancient and prestigious and privileged university (especially Oxford University or Cambridge University)',
   -inf),
  ('county, NN -- (United Kingdom) a region created by territorial division for the purpose of local government',
   -inf),
  ('stockbroker belt, NN -- a wealthy residential suburb', -inf),
  ('lucky dip, NN -- a game in which prizes (e.g., candies or coins) are concealed in a container and for a small sum a player can draw one out at random',
   -inf),
  ('potman, NN -- a worker in an inn or public house who serves customers and does various chores',
   -inf),
  ('eccles cake, NN -- a flat round cake of sweetened pastry filled with dried fruit',
   -inf),
  ('ir, NN -- a board of the British government that administers and collects major direct taxes',
   -inf),
  ('victoria sponge, NN -- a cake consisting of two layers of sponge cake with a jelly filli

In [482]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

32 predict tail: ali kazemaini | birthplace | tehran
34 predict tail: ashta, maharashtra | instance of | human settlement
37 predict tail: roy shaw 0 | instance of | human being
40 predict tail: t. canby jones | has surname | jones (family name)
45 predict tail: naveen kumar | instance of | human being
46 predict tail: barlow respiratory hospital | host country | united states of america
48 predict tail: camiling | office held by head of government | mayor
51 predict tail: hazel soan | instance of | human being
52 predict tail: efrain herrera | sport played | association football
53 predict tail: oluf munck | instance of | human being
54 predict tail: thomas gilchrist | instance of | human being
58 predict tail: desmoplastic fibroma | subclass of | fibroma


('count', 12)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]